In [1]:
import re
from dataclasses import dataclass
from collections import defaultdict
from tqdm.notebook import tqdm

In [2]:
pattern = r"(\w+)-to-(\w+) map:"

In [3]:
@dataclass
class Range:
    destination_start: int
    source_start: int
    length: int

In [4]:
data = open("input/05").read().splitlines()

In [5]:
seeds = list(map(int, data[0].split("seeds: ")[1].split(" ")))

In [6]:
maps = defaultdict(list)
cur_map = None
keys = []
for line in data[2:]:
    match = re.match(pattern, line)
    if match:
        from_, to = match.groups()
        keys.append((from_, to))
    else:
        if not line:
            continue
        nums = list(map(int, line.split(" ")))
        maps[(from_, to)].append(Range(*nums))

In [7]:
def check_seed(seed):
    cur_location = seed
    for key in keys:   
        for entry in maps[key]:        
            if cur_location >= entry.source_start and cur_location <= (entry.source_start + entry.length - 1):
                cur_location = cur_location - (entry.source_start - entry.destination_start)
                break
    
    return cur_location

# Part 1

In [8]:
part1_seeds = seeds

In [9]:
best_location = None
for seed in part1_seeds:
    cp = check_seed(seed)
    if not best_location:
        best_location = cp
    else:
        best_location = min(best_location, cp)

print("Answer #1:", best_location)

Answer #1: 806029445


# Part 2

In [10]:
def check_reverse_seed(goal):
    cur_location = goal
    for key in keys[::-1]:
        for entry in maps[key]:            
            if cur_location >= entry.destination_start and cur_location <= (entry.destination_start + entry.length - 1):                
                cur_location = cur_location - (entry.destination_start - entry.source_start)
                break
    return cur_location

In [11]:
ranges = []
for i in range(0, len(seeds), 2):
    start_seed = seeds[i]
    range_seed = seeds[i + 1]
    ranges.append([start_seed, start_seed + range_seed - 1])

## Quicker solution, not optimal

In [12]:
%%time
i = 1
while True:
    found = False
    seed = check_reverse_seed(i)
    for start, end in ranges:
        if seed >= start and seed <= end:
            found = True
            break
    if found:
        break
    i += 1
        
print("Answer #2:", i)
print()

Answer #2: 59370572

CPU times: user 7min 44s, sys: 1.89 s, total: 7min 46s
Wall time: 7min 46s


## Brute force solution, around 4 hours

In [13]:
%%time
best_location = None
for i in tqdm(range(0, len(seeds), 2)):
    start_seed = seeds[i]
    range_seed = seeds[i + 1]    
    for i in range(range_seed):
        cp = check_seed(start_seed + i)
        if not best_location:
            best_location = cp
        else:
            best_location = min(best_location, cp)

print("Answer #2:", best_location)
print()

  0%|          | 0/10 [00:00<?, ?it/s]

Answer #2: 59370572

CPU times: user 3h 43min 31s, sys: 53.4 s, total: 3h 44min 24s
Wall time: 3h 44min 44s
